Creating a Gauge Chart using xlsxwriter and python

An example in the xlsxwriter documentation:
https://xlsxwriter.readthedocs.io/example_chart_gauge.html

In [ ]:
#pulling in data for the gauge chart
import pandas_datareader.data as data
import pandas as pd
import datetime

umcsent_df = data.DataReader("UMCSENT", 'fred', datetime.datetime(2000,12,31))
umcsent_df['perc'] = (umcsent_df['UMCSENT'] / umcsent_df['UMCSENT'].max()) * 100
umcsent_df['xlsx_date'] = umcsent_df.index - datetime.datetime(1899, 12, 31)
print(umcsent_df.tail(10))

current_date = umcsent_df.index[-1].strftime("%B") + '-' + umcsent_df.index[-1].strftime("%Y")
current_value = umcsent_df['perc'].iloc[-1] / 100

            UMCSENT       perc  xlsx_date
DATE                                     
2020-06-01     78.1  75.240848 43982 days
2020-07-01     72.5  69.845857 44012 days
2020-08-01     74.1  71.387283 44043 days
2020-09-01     80.4  77.456647 44074 days
2020-10-01     81.8  78.805395 44104 days
2020-11-01     76.9  74.084778 44135 days
2020-12-01     80.7  77.745665 44165 days
2021-01-01     79.0  76.107900 44196 days
2021-02-01     76.8  73.988439 44227 days
2021-03-01     84.9  81.791908 44255 days


In [ ]:
!pip install xlsxwriter
import xlsxwriter

workbook = xlsxwriter.Workbook('gaugeChartXLSX.xlsx')
worksheet = workbook.add_worksheet()

chart_doughnut = workbook.add_chart({'type': 'doughnut'})
chart_pie = workbook.add_chart({'type': 'pie'})

# Making the doughnut chart. IMO - best way to do is to sum it to 360 (180 is blank)
chart_doughnut.add_series({
    'values': '{36,36,36,36,36,180}',
    'points': [
               {'fill': {'color': 'red'}},
               {'fill': {'color': 'orange'}},
               {'fill': {'color': 'yellow'}},
               {'fill': {'color': 'lime'}},
               {'fill': {'color': 'green'}},
               {'fill': {'none': True}},
    ]
})

#Rotate the chart so it's above horizontal and turning off the legend/border
chart_doughnut.set_rotation(270)
chart_doughnut.set_chartarea({
    'border': {'none': True},
    'fill': {'none': True},
})
chart_doughnut.set_legend({'none': True})

#Pie Chart - setting the needle on the gauge chart. Sum it to 180, e.g., {89, 1, 90}.
point_a = round(current_value * 89)
point_c = round((1-current_value) * 89) + 90
cust_labels = [
    {'value': ''},
    {'value': '%d' % (round(current_value*100, 1))},
    {'value': ''},
]
chart_pie.add_series({
    'name': ' ',
    'values': '{%d,1,%d}' % (point_a, point_c),
    'points': [
        {'fill': {'none': True}},
        {'fill': {'color': 'black'}},
        {'fill': {'none': True}}],
    'data_labels':{ 
         'border': {'color': 'red'},
         'fill':   {'color': 'yellow'},
         'custom': cust_labels},
})

# Rotate the pie chart too
chart_pie.set_rotation(270)

#combine charts / give title / inset into worksheet
chart_doughnut.combine(chart_pie)
chart_doughnut.set_title({'name': 'UMCSENT %s' % (current_date)})
worksheet.insert_chart('A1', chart_doughnut)

#adding simpleline chart and data for context for the user
dateFormat = workbook.add_format({'num_format': 'mmm-yy'})
chart = workbook.add_chart({'type': 'line'})
chart.set_title({'name': 'UMCSENT (Jan-2001 to %s' % (current_date)})
worksheet.write_column('H2', umcsent_df['xlsx_date'], dateFormat)
worksheet.write_column('I2', umcsent_df['UMCSENT'])
chart.add_series({
    'values': '=Sheet1!$I$2:$I$%d' % (len(umcsent_df)),
    'categories': '=Sheet1!$H$2:$H$%d' % (len(umcsent_df)),
})
worksheet.insert_chart('K2', chart)

workbook.close()